In [214]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pandas.api.types import is_numeric_dtype
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score, roc_curve, auc

In [215]:
# Import the CSV file named 'water_potability.csv' into a pandas DataFrame
df = pd.read_csv("water_potability.csv")

# Display the first five rows of the DataFrame to preview the data
df.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0


In [216]:
# Display a concise summary of the DataFrame, including:
# - Number of rows and columns
# - Column names and data types
# - Non-null counts for each column (to check missing values)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3276 entries, 0 to 3275
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ph               2785 non-null   float64
 1   Hardness         3276 non-null   float64
 2   Solids           3276 non-null   float64
 3   Chloramines      3276 non-null   float64
 4   Sulfate          2495 non-null   float64
 5   Conductivity     3276 non-null   float64
 6   Organic_carbon   3276 non-null   float64
 7   Trihalomethanes  3114 non-null   float64
 8   Turbidity        3276 non-null   float64
 9   Potability       3276 non-null   int64  
dtypes: float64(9), int64(1)
memory usage: 256.1 KB


In [217]:
# Get the dimensions of the DataFrame `df` as (number of rows, number of columns)
df.shape

(3276, 10)

In [218]:
# Summary statistics (count, mean, std, min, max, quartiles) of numeric columns
df.describe()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
count,2785.000000,3276.000000,3276.000000,3276.000000,2495.000000,3276.000000,3276.000000,3114.000000,3276.000000,3276.000000
mean,7.080795,196.369496,22014.092526,7.122277,333.775777,426.205111,14.284970,66.396293,3.966786,0.390110
std,1.594320,32.879761,8768.570828,1.583085,41.416840,80.824064,3.308162,16.175008,0.780382,0.487849
min,0.000000,47.432000,320.942611,0.352000,129.000000,181.483754,2.200000,0.738000,1.450000,0.000000
25%,6.093092,176.850538,15666.690297,6.127421,307.699498,365.734414,12.065801,55.844536,3.439711,0.000000
50%,7.036752,196.967627,20927.833607,7.130299,333.073546,421.884968,14.218338,66.622485,3.955028,0.000000
75%,8.062066,216.667456,27332.762127,8.114887,359.950170,481.792304,16.557652,77.337473,4.500320,1.000000
max,14.000000,323.124000,61227.196008,13.127000,481.030642,753.342620,28.300000,124.000000,6.739000,1.000000


In [219]:
# Print the count of missing (NaN) values in each column of the DataFrame
print(df.isna().sum())

ph                 491
Hardness             0
Solids               0
Chloramines          0
Sulfate            781
Conductivity         0
Organic_carbon       0
Trihalomethanes    162
Turbidity            0
Potability           0
dtype: int64


In [220]:
# Print the total number of duplicate rows in the DataFrame
print(df.duplicated().sum())

0


In [221]:
# Initialize a dictionary to store the outlier count for each column
count_outlier = {}

# Loop through each column in the DataFrame
for col in df.columns:
    # Calculate the 25th percentile (Q1) of the column
    q1 = df[col].quantile(.25)
    # Calculate the 75th percentile (Q3) of the column
    q3 = df[col].quantile(.75)

    # Compute the interquartile range (IQR)
    iqr = q3 - q1

    # Define the upper whisker for outlier detection
    upper_whisker = q3 + 1.5 * iqr
    # Define the lower whisker for outlier detection
    lower_whisker = q1 - 1.5 * iqr

    # Identify outliers as values outside the whiskers
    counts = (df[col] > upper_whisker) | (df[col] < lower_whisker)
    # Store the count of outliers for this column
    count_outlier[col] = counts.sum()

# Convert the outlier counts dictionary into a DataFrame for display
outlier_df = pd.DataFrame(
    {
        'Columns': count_outlier.keys(),       # Column names
        'Outlier Count': count_outlier.values() # Corresponding outlier counts
    }
)

# Display the DataFrame showing outlier counts per column
outlier_df

,Columns,Outlier Count
0,ph,46
1,Hardness,83
2,Solids,47
3,Chloramines,61
4,Sulfate,41
5,Conductivity,11
6,Organic_carbon,25
7,Trihalomethanes,33
8,Turbidity,19
9,Potability,0


In [222]:
'''def outlier(df, col2):
    # Calculate the 25th percentile (Q1) of the column
    q1 = df[col2].quantile(.25)
    # Calculate the 75th percentile (Q3) of the column
    q3 = df[col2].quantile(.75)

    # Compute the interquartile range (IQR)
    iqr = q3 - q1

    # Define the upper whisker for outlier detection
    upper_whisker = q3 + 1.5 * iqr
    # Define the lower whisker for outlier detection
    lower_whisker = q1 - 1.5 * iqr

    df = df[(df[col2] <= upper_whisker) & (df[col2] >= lower_whisker)]

    return df'''

'def outlier(df, col2):\n    # Calculate the 25th percentile (Q1) of the column\n    q1 = df[col2].quantile(.25)\n    # Calculate the 75th percentile (Q3) of the column\n    q3 = df[col2].quantile(.75)\n\n    # Compute the interquartile range (IQR)\n    iqr = q3 - q1\n\n    # Define the upper whisker for outlier detection\n    upper_whisker = q3 + 1.5 * iqr\n    # Define the lower whisker for outlier detection\n    lower_whisker = q1 - 1.5 * iqr\n\n    df = df[(df[col2] <= upper_whisker) & (df[col2] >= lower_whisker)]\n\n    return df'

In [223]:
'''df_clean = df.copy()
for col3 in df_clean.columns:
    df_clean = outlier(df_clean, col3)
df = df_clean.copy()'''

'df_clean = df.copy()\nfor col3 in df_clean.columns:\n    df_clean = outlier(df_clean, col3)\ndf = df_clean.copy()'

In [224]:
'''def outlier_zscore(df, col4):
    mean_val = df[col4].mean()
    std_val = df[col4].std()

    z_score = (df[col4] - mean_val) / std_val

    df = df[(z_score <= 3) & (z_score >= -3)]
    return df'''

'def outlier_zscore(df, col4):\n    mean_val = df[col4].mean()\n    std_val = df[col4].std()\n\n    z_score = (df[col4] - mean_val) / std_val\n\n    df = df[(z_score <= 3) & (z_score >= -3)]\n    return df'

In [225]:
'''df_clean2 = df.copy()
for col5 in df_clean2.columns:
    df_clean2 = outlier_zscore(df_clean2, col5)
df = df_clean2.copy()'''

'df_clean2 = df.copy()\nfor col5 in df_clean2.columns:\n    df_clean2 = outlier_zscore(df_clean2, col5)\ndf = df_clean2.copy()'

In [226]:

df_original = df.copy()  # মূল dataset safe রাখি

iqr_factors = [1.0, 1.5, 2.0, 2.5, 3.0]  # বিভিন্ন factor test
results = []

for k in iqr_factors:
    df_temp = df_original.copy()
    
    for col in df_temp.select_dtypes(include='number').columns:
        q1 = df_temp[col].quantile(0.25)
        q3 = df_temp[col].quantile(0.75)
        iqr = q3 - q1
        lower = q1 - k*iqr
        upper = q3 + k*iqr
        
        # Filter values outside the whiskers
        df_temp = df_temp[(df_temp[col] >= lower) & (df_temp[col] <= upper)]
    
    # Row loss percentage
    rows_removed = len(df_original) - len(df_temp)
    row_loss_percent = rows_removed / len(df_original) * 100
    
    results.append((k, len(df_temp), row_loss_percent))

# Convert to DataFrame for easy view
optimum_df = pd.DataFrame(results, columns=['IQR_factor', 'Remaining_rows', 'Row_loss_percent'])
optimum_df


,IQR_factor,Remaining_rows,Row_loss_percent
0,1.0,1230,62.454212
1,1.5,1791,45.329670
2,2.0,1966,39.987790
3,2.5,2003,38.858364
4,3.0,2009,38.675214
